# Chapter 24: Dynamic Attributes and Metaprogramming

Python gives you fine-grained control over attribute access through a family of dunder
methods: `__getattr__`, `__getattribute__`, `__setattr__`, and `__delattr__`. These hooks
let you intercept how attributes are read, written, and deleted, enabling patterns like
proxy objects, lazy loading, validation, and dynamic configuration.

## Topics Covered
- **`__getattr__`**: Called when normal attribute lookup fails
- **`__getattribute__`**: Called on every attribute access (use carefully)
- **`__setattr__`**: Intercepting attribute assignment and validation
- **`__delattr__`**: Intercepting attribute deletion
- **Proxy objects** with `__getattr__`
- **Lazy attribute loading** patterns
- **Property factories**: Creating properties dynamically
- **Practical**: A configuration object with dot notation access

## `__getattr__`: Fallback for Missing Attributes

`__getattr__` is called **only** when the normal attribute lookup mechanism fails --
that is, when the attribute is not found in the instance `__dict__`, the class, or any
base class. This makes it safe and efficient: it does not interfere with attributes
that actually exist.

In [ ]:
class FlexibleObject:
    """Demonstrates __getattr__ as a fallback for missing attributes."""

    def __init__(self, name: str) -> None:
        self.name = name  # This is a real attribute

    def __getattr__(self, attr: str) -> str:
        """Called ONLY when normal lookup fails."""
        print(f"  __getattr__ called for: {attr!r}")
        return f"<default for {attr}>"


obj = FlexibleObject("demo")

# Accessing a real attribute -- __getattr__ is NOT called
print(f"obj.name = {obj.name!r}")

# Accessing a missing attribute -- __getattr__ IS called
print(f"obj.color = {obj.color!r}")
print(f"obj.size = {obj.size!r}")

# hasattr uses __getattr__ under the hood
print(f"\nhasattr(obj, 'name'): {hasattr(obj, 'name')}")
print(f"hasattr(obj, 'anything'): {hasattr(obj, 'anything')}")

## `__getattribute__`: Intercepting Every Attribute Access

`__getattribute__` is called on **every** attribute access, even for attributes that
exist. This is powerful but dangerous -- if implemented incorrectly it can cause infinite
recursion. Always use `super().__getattribute__()` or `object.__getattribute__()` to
perform the actual lookup inside your override.

In [ ]:
class AuditedAccess:
    """Logs every attribute access using __getattribute__."""

    def __init__(self, x: int, y: int) -> None:
        self.x = x
        self.y = y
        self._access_log: list[str] = []

    def __getattribute__(self, name: str) -> object:
        """Called on EVERY attribute access, including existing ones."""
        # Must use object.__getattribute__ to avoid infinite recursion
        if not name.startswith("_"):
            log = object.__getattribute__(self, "_access_log")
            log.append(name)
        return object.__getattribute__(self, name)

    def get_log(self) -> list[str]:
        return self._access_log


audited = AuditedAccess(10, 20)
print(f"audited.x = {audited.x}")
print(f"audited.y = {audited.y}")
print(f"audited.x = {audited.x}")

# The access log recorded every attribute read
print(f"\nAccess log: {audited.get_log()}")

## `__setattr__`: Intercepting Attribute Assignment

`__setattr__` is called on **every** attribute assignment, including during `__init__`.
This makes it ideal for validation but requires care: use `object.__setattr__()` or
`self.__dict__[name] = value` to actually store the attribute without recursion.

In [ ]:
class ValidatedPoint:
    """Validates that x and y are always numeric."""

    def __init__(self, x: float, y: float) -> None:
        self.x = x  # Goes through __setattr__
        self.y = y

    def __setattr__(self, name: str, value: object) -> None:
        if name in ("x", "y"):
            if not isinstance(value, (int, float)):
                raise TypeError(
                    f"{name!r} must be numeric, got {type(value).__name__}"
                )
            # Use object.__setattr__ to actually store the value
            object.__setattr__(self, name, float(value))
        else:
            object.__setattr__(self, name, value)


p = ValidatedPoint(3, 4)
print(f"p.x = {p.x}, p.y = {p.y}")

p.x = 10  # Validated and converted to float
print(f"After p.x = 10: p.x = {p.x}")

# Invalid assignment raises TypeError
try:
    p.x = "hello"
except TypeError as e:
    print(f"\nTypeError: {e}")

# Other attributes are accepted without validation
p.label = "origin"
print(f"p.label = {p.label!r}")

## `__delattr__`: Intercepting Attribute Deletion

`__delattr__` is called when `del obj.attr` is executed. You can use it to
prevent deletion of critical attributes or to perform cleanup.

In [ ]:
class ProtectedAttributes:
    """Prevents deletion of protected attributes."""

    _protected: frozenset[str] = frozenset({"id", "created_at"})

    def __init__(self, id: int, created_at: str, name: str) -> None:
        self.id = id
        self.created_at = created_at
        self.name = name

    def __delattr__(self, name: str) -> None:
        if name in self._protected:
            raise AttributeError(
                f"Cannot delete protected attribute {name!r}"
            )
        print(f"  Deleting attribute {name!r}")
        object.__delattr__(self, name)


record = ProtectedAttributes(42, "2025-01-01", "Alice")
print(f"record.name = {record.name!r}")

# Deleting an unprotected attribute works
del record.name
print(f"hasattr(record, 'name'): {hasattr(record, 'name')}")

# Deleting a protected attribute is blocked
try:
    del record.id
except AttributeError as e:
    print(f"\nAttributeError: {e}")

print(f"record.id still = {record.id}")

## Building Proxy Objects with `__getattr__`

A **proxy** wraps another object and forwards attribute access to it. `__getattr__`
is perfect for this because it only fires when the proxy itself does not have the
requested attribute, naturally delegating to the wrapped object.

In [ ]:
from typing import Any


class LoggingProxy:
    """Wraps any object and logs all attribute access and method calls."""

    def __init__(self, target: Any) -> None:
        # Store target in __dict__ directly to avoid triggering __setattr__
        object.__setattr__(self, "_target", target)
        object.__setattr__(self, "_log", [])

    def __getattr__(self, name: str) -> Any:
        target = object.__getattribute__(self, "_target")
        log = object.__getattribute__(self, "_log")

        attr = getattr(target, name)
        if callable(attr):
            def logged_method(*args: Any, **kwargs: Any) -> Any:
                log.append(f"CALL: {name}({args}, {kwargs})")
                return attr(*args, **kwargs)
            return logged_method
        else:
            log.append(f"GET: {name} -> {attr!r}")
            return attr


# Proxy a regular list
real_list: list[int] = [1, 2, 3]
proxy = LoggingProxy(real_list)

# All operations are forwarded to the real list
proxy.append(4)
proxy.extend([5, 6])
length = proxy.__len__()

print(f"Real list: {real_list}")
print(f"\nProxy log:")
for entry in proxy._log:
    print(f"  {entry}")

## Lazy Attribute Loading Patterns

Lazy loading defers expensive computation until the attribute is first accessed.
You can implement this with `__getattr__` -- the attribute starts missing, gets
computed on first access, and then is stored so `__getattr__` is never called again
for that attribute.

In [ ]:
import time
from typing import Any


class LazyDataLoader:
    """Loads expensive data on first access, then caches it."""

    def __init__(self, source: str) -> None:
        self.source = source
        # Note: 'data' and 'summary' are NOT set here -- they are lazy

    def _load_data(self) -> list[dict[str, Any]]:
        """Simulate expensive data loading."""
        print(f"  Loading data from {self.source!r} (expensive operation)...")
        time.sleep(0.1)  # Simulate I/O delay
        return [
            {"id": 1, "value": 100},
            {"id": 2, "value": 200},
            {"id": 3, "value": 300},
        ]

    def _compute_summary(self) -> dict[str, float]:
        """Compute a summary from the loaded data."""
        print("  Computing summary (depends on data)...")
        values = [row["value"] for row in self.data]
        return {"count": len(values), "total": sum(values), "mean": sum(values) / len(values)}

    def __getattr__(self, name: str) -> Any:
        """Lazy-load attributes on first access."""
        loaders: dict[str, Any] = {
            "data": self._load_data,
            "summary": self._compute_summary,
        }
        if name in loaders:
            value = loaders[name]()
            # Store in __dict__ so __getattr__ is not called again
            self.__dict__[name] = value
            return value
        raise AttributeError(f"{type(self).__name__!r} has no attribute {name!r}")


loader = LazyDataLoader("database")
print("Created loader -- no data loaded yet")
print(f"loader.__dict__ keys: {list(loader.__dict__.keys())}\n")

# First access triggers loading
print(f"loader.data = {loader.data}\n")

# Second access is instant -- cached in __dict__
print("Accessing again (should be cached):")
print(f"loader.data = {loader.data}\n")

# Summary depends on data (which is already loaded)
print(f"loader.summary = {loader.summary}")

print(f"\nFinal __dict__ keys: {list(loader.__dict__.keys())}")

## Property Factories: Creating Properties Dynamically

You can create `property` objects programmatically and attach them to classes at
runtime. This is useful when you want many similar properties with the same pattern
(e.g., validated, type-checked, or logged).

In [ ]:
from typing import Any


def make_validated_property(
    name: str,
    expected_type: type,
    min_value: float | None = None,
    max_value: float | None = None,
) -> property:
    """Factory that creates a validated property with range checking."""
    storage_name = f"_{name}"

    def getter(self: Any) -> Any:
        return getattr(self, storage_name, None)

    def setter(self: Any, value: Any) -> None:
        if not isinstance(value, expected_type):
            raise TypeError(
                f"{name!r} must be {expected_type.__name__}, got {type(value).__name__}"
            )
        if min_value is not None and value < min_value:
            raise ValueError(f"{name!r} must be >= {min_value}, got {value}")
        if max_value is not None and value > max_value:
            raise ValueError(f"{name!r} must be <= {max_value}, got {value}")
        setattr(self, storage_name, value)

    return property(getter, setter, doc=f"Validated {expected_type.__name__} property")


class Sensor:
    """A sensor with dynamically created validated properties."""
    temperature = make_validated_property("temperature", (int, float), -273.15, 1000.0)
    humidity = make_validated_property("humidity", (int, float), 0.0, 100.0)
    label = make_validated_property("label", str)


sensor = Sensor()
sensor.temperature = 22.5
sensor.humidity = 65
sensor.label = "living_room"

print(f"temperature: {sensor.temperature}")
print(f"humidity: {sensor.humidity}")
print(f"label: {sensor.label!r}")

# Validation in action
for desc, action in [
    ("temperature = 'hot'", lambda: setattr(sensor, "temperature", "hot")),
    ("temperature = -300", lambda: setattr(sensor, "temperature", -300)),
    ("humidity = 150", lambda: setattr(sensor, "humidity", 150)),
]:
    try:
        action()
    except (TypeError, ValueError) as e:
        print(f"\n{desc} -> {type(e).__name__}: {e}")

## Practical: Configuration Object with Dot Notation Access

A common use case for dynamic attributes is building a configuration object that
allows nested dot notation (`config.database.host`) backed by a plain dictionary.
This combines `__getattr__`, `__setattr__`, and `__delattr__` into a cohesive pattern.

In [ ]:
from typing import Any


class DotConfig:
    """Configuration object that supports nested dot notation access.

    Wraps a dictionary so you can write `config.database.host` instead
    of `config['database']['host']`.
    """

    def __init__(self, data: dict[str, Any] | None = None, **kwargs: Any) -> None:
        # Use object.__setattr__ to bypass our custom __setattr__
        object.__setattr__(self, "_data", {})
        source = data if data is not None else kwargs
        for key, value in source.items():
            self[key] = value  # Uses __setitem__ -> recursion for nested dicts

    def __getattr__(self, name: str) -> Any:
        """Access config values with dot notation."""
        try:
            return self._data[name]
        except KeyError:
            raise AttributeError(f"No config key {name!r}") from None

    def __setattr__(self, name: str, value: Any) -> None:
        """Set config values with dot notation."""
        self[name] = value

    def __delattr__(self, name: str) -> None:
        """Delete config values with dot notation."""
        try:
            del self._data[name]
        except KeyError:
            raise AttributeError(f"No config key {name!r}") from None

    def __setitem__(self, key: str, value: Any) -> None:
        """Recursively wrap nested dicts as DotConfig."""
        if isinstance(value, dict):
            value = DotConfig(value)
        self._data[key] = value

    def __contains__(self, key: str) -> bool:
        return key in self._data

    def __repr__(self) -> str:
        return f"DotConfig({self._data!r})"

    def to_dict(self) -> dict[str, Any]:
        """Convert back to a plain dictionary."""
        result: dict[str, Any] = {}
        for key, value in self._data.items():
            result[key] = value.to_dict() if isinstance(value, DotConfig) else value
        return result


# Build config from a nested dictionary
raw_config: dict[str, Any] = {
    "app_name": "my_service",
    "debug": True,
    "database": {
        "host": "localhost",
        "port": 5432,
        "credentials": {
            "user": "admin",
            "password": "secret",
        },
    },
    "features": {
        "enable_cache": True,
        "max_retries": 3,
    },
}

config = DotConfig(raw_config)

# Dot notation access, including deeply nested values
print(f"App name:     {config.app_name!r}")
print(f"Debug:        {config.debug}")
print(f"DB host:      {config.database.host!r}")
print(f"DB port:      {config.database.port}")
print(f"DB user:      {config.database.credentials.user!r}")
print(f"Cache on:     {config.features.enable_cache}")

In [ ]:
# Mutating the config with dot notation
config.database.port = 3306
config.database.credentials.password = "new_secret"
config.features.max_retries = 5

# Adding new nested sections
config.logging = {"level": "INFO", "file": "/var/log/app.log"}

print(f"Updated DB port:     {config.database.port}")
print(f"Updated password:    {config.database.credentials.password!r}")
print(f"Updated retries:     {config.features.max_retries}")
print(f"New log level:       {config.logging.level!r}")

# Deleting a config key
del config.debug
print(f"\n'debug' in config: {'debug' in config}")

# Convert back to plain dict
print(f"\nAs dict: {config.to_dict()}")

## Summary

### Key Takeaways

| Hook | When Called | Use Case |
|------|------------|----------|
| `__getattr__` | Attribute not found by normal lookup | Defaults, proxies, lazy loading |
| `__getattribute__` | Every attribute access | Auditing, access control (use sparingly) |
| `__setattr__` | Every attribute assignment | Validation, logging, read-only attrs |
| `__delattr__` | Every `del obj.attr` | Protecting critical attributes |

### Best Practices
- Prefer `__getattr__` over `__getattribute__` -- it is safer and more efficient
- In `__getattribute__` and `__setattr__`, use `object.__getattribute__()` or
  `object.__setattr__()` to avoid infinite recursion
- Cache lazily loaded values in `__dict__` so the hook is only called once
- Property factories reduce boilerplate when many attributes share the same validation
- Proxy objects with `__getattr__` are great for logging, caching, and access control
- For nested dict access, a `DotConfig` wrapper can make configuration code much cleaner